In [327]:
from math import sin, log, cos
from abc import ABC, abstractmethod

In [328]:
def f(x):
    return (3*x**2 + x) * sin(x)

In [329]:
def package(maybe_expression):
    if isinstance(maybe_expression,Expression):
        return maybe_expression
    elif isinstance(maybe_expression,int) or isinstance(maybe_expression,float):
        return Number(maybe_expression)
    else:
        raise ValueError("can't convert {} to expression.".format(maybe_expression))

In [330]:
class Expression(ABC):
    @abstractmethod
    def evaluate(self,**bindings):
        pass
    @abstractmethod
    def expand(self):
        pass
    def __repr__(self):
        return self.display()
    def __add__(self, other):
        return Sum(self,package(other))
    def __sub__(self,other):
        return Difference(self,package(other))
    def __mul__(self,other):
        return Product(self,package(other))
    def __rmul__(self,other):
        return Product(package(other),self)
    def __truediv__(self,other):
        return Quotient(self,package(other))
    def __pow__(self,other):
        return Power(self,package(other))

In [331]:
class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent
    def evaluate(self, **bindings):
        return self.base.evaluate(**bindings) ** self.exponent.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Power({},{})".format(self.base.display(),self.exponent.display())

In [332]:
class Number(Expression):
    def __init__(self,number):
        self.number = number
    def evaluate(self,**bindings):
        return self.number
    def expand(self):
        return self
    def display(self):
        return "Number({})".format(self.number)

In [333]:
class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol
    def evaluate(self,**bindings):
        try:
            return bindings[self.symbol]
        except:
            raise KeyError("Variable '{}' is not bound.".format(self.symbol))
    def expand(self):
        return self
    def display(self):
        return "Variable(\"{}\")".format(self.symbol)

In [334]:
class Product(Expression):
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self,**bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)
    def expand(self):
        expanded1 = self.exp1.expand()
        expanded2 = self.exp2.expand()
        if isinstance(expanded1,Sum):
            return Sum(*[Product(e,expanded2).expand() for e in expanded1.exps])
        elif isinstance(expanded2,Sum):
            return Sum(*[Product(expanded1,e) for e in expanded2.exps])
        else:
            return Product(expanded1,expanded2)
    def display(self):
        return "Product({},{})".format(self.exp1.display(),self.exp2.display())

In [335]:
class Sum(Expression):
    def __init__(self,*exps):
        self.exps = exps
    def evaluate(self, **bindings):
        return sum([exp.evaluate(**bindings) for exp in self.exps])
    def expand(self):
        return Sum(*[exp.expand() for exp in self.exps])
    def display(self):
        return "Sum({})".format(",".join([e.display() for e in self.exps]))

In [336]:
class Function():
    def __init__(self,name):
        self.name = name

In [337]:
class Apply(Expression):
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument
    def evaluate(self,**bindings):
        return _function_bindings[self.function.name](self.argument.evaluate(**bindings))
    def expand(self):
        return Apply(self.function,self.argument.expand())
    def display(self):
        return "Apply(Function(\"{}\"),{})".format(self.function.name, self.argument.display())

In [338]:
# (3*x**2 + x) * sin(x)
f_expression = Product(
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2)
                        )
                    ),
                    Variable("x")
                ),
                Apply(
                    Function("sin"),
                    Variable("x")
                )
            )

In [339]:
def log_function(y,z):
    return log(y**z)

In [340]:
log_expression = Apply(Function("ln"),Power(Variable("y"),Variable("z")))

In [341]:
class Quotient():
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator
    def evaluate(self, **bindings):
        return self.numerator.evaluate(**bindings) / self.denominator.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Quotient({},{})".format(self.numerator.display(),self.denominator.display())

In [342]:
# a+b / 2
quotient_expression = Quotient(Sum(Variable("a"),Variable("b")),Number(2))

In [343]:
class Difference():
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) - self.exp2.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Difference({},{})".format(self.exp1.display(), self.exp2.display())

In [344]:
# b**2 - 4*a*c
difference_expression = Difference(
                            Power(
                                Variable("b"),
                                Number(2)
                            ),
                            Product(
                                Product(
                                    Number(4),
                                    Variable("a")
                                ),
                                Variable("c")
                            )
                        )

In [345]:
class Negative():
    def __init__(self,exp):
        self.exp = exp
    def evaluate(self, **bindings):
        return - self.exp.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Negative({})".format(self.exp.display())

In [346]:
# -(x**2 + y)
negative_expression = Negative(Sum(Power(Variable("x"),Number(2)),Variable("y")))

In [347]:
# (-b +- sprt(b**2 - 4*a*c) / 2**a)
sqrt_expression = Quotient(
                    Sum(
                        Negative(Variable("b")),
                        Apply(
                            Function("sqrt"),
                            Difference(
                                Power(
                                    Variable("b"),
                                    Number(2)
                                ),
                                Product(
                                    Number(4),
                                    Product(
                                        Variable("a"),
                                        Variable("c")
                                    )
                                )
                            )
                        )
                    ),
                    Product(
                        Number(2),
                        Variable("a")
                    )
                )

In [348]:
def distinct_variables(exp):
    if isinstance(exp, Variable):
        return set(exp.symbol)
    elif isinstance(exp, Number):
        return set()
    elif isinstance(exp, Sum):
        return set().union(*[distinct_variables(exp) for exp in exp.exps])
    elif isinstance(exp, Product):
        return distinct_variables(exp.exp1).union(distinct_variables(exp.exp2))
    elif isinstance(exp, Power):
        return distinct_variables(exp.base).union(distinct_variables(exp.exponent))
    elif isinstance(exp, Apply):
        return distinct_variables(exp.argument)
    else:
        raise TypeError("Not a valid expression.")

In [349]:
distinct_variables(f_expression)

{'x'}

In [350]:
_function_bindings = {
    "sin": sin,
    "cos": cos,
    "ln": log
}

In [351]:
f_expression.evaluate(x=5)

-76.71394197305108

In [352]:
f(5)

-76.71394197305108

In [353]:
Y = Variable('y')
Z = Variable('z')
A = Variable('a')
B = Variable('b')

In [354]:
Product(Sum(A,B),Sum(Y,Z))

Product(Sum(Variable("a"),Variable("b")),Sum(Variable("y"),Variable("z")))

In [355]:
f_expression.expand()

Sum(Product(Product(Number(3),Power(Variable("x"),Number(2))),Apply(Function("sin"),Variable("x"))),Product(Variable("x"),Apply(Function("sin"),Variable("x"))))